# ========================================================
# Dark URL Detection
# Adrien Manciet - Thibault Sourdeval
# ========================================================

In [ ]:
#Introduction

Ce dataset est un ensemble d'url qui sont labellisés. Si le label vaut 1, l'url est dangereux, si il vaut -1, il ne l'est pas. L'objectif sera de faire un algorithme de classification des url en apprenant sur le dataset disponible. 

# =========================
# Partie 1 : Phase d'exploration
# =========================

In [ ]:
#Code

Nous notons que les fichiers de données sont sous la forme de matrices sparse. Cela signifie que seules les valeurs non nulles sont gardées en mémoire. 
Cela permet d'épargner des erreurs de mémoire. 

Le fichier features contient des numéros qui semblent correspondre à des subdivisions de l'url contenant en blocs. Exemple : la première ligne du fichier features affiche 4, ce qui pourrait correspondre aux quatres premiers caractères de l'url 'http'. 

Nous codons une fonction de prévisualisation pour mieux comprendre la structure des données en les transformant en un dataframe. Pour la suite, 
nous resterons dans le format de données initial.

**Fonction de prévisualisation** 

In [1]:
from sklearn.datasets import load_svmlight_file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

def preview_data(day, nb_lines, nb_cols, random = True):
    # Si random est laissé tel quel, une valeur au hasard 
    # est prise pour la première ligne et la première colonne à afficher

    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)
    print(X.shape)

    if random == True:
        start_line = np.random.randint(0, len(y)-nb_lines)
        start_col = np.random.randint(0,X.shape[1])
    else : 
        start_line = int(input("Première ligne à afficher : "))
        start_col = int(input("Première colonne à afficher : "))
    
    label_list = []
    for i in range(start_col, start_col+nb_cols):
        label_list.append(i)

    X_df = pd.DataFrame(X[start_line: start_line+nb_lines, start_col: start_col+nb_cols].toarray(), columns=label_list)
    y_df = pd.DataFrame(y[start_line: start_line+nb_lines], columns=['label'])

    data = pd.concat([X_df, y_df], axis=1)
    return data

preview_data(17, 10, 10, random=False)


**Visualisations grahiques** 

In [ ]:
def scatter_plot(day, feature_x, feature_y): 
    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)

    X = X[0:1000,:1000].toarray()
    plt.figure()
    plt.grid(alpha=0.2)
    sc = plt.scatter(X[:,feature_x], X[:,feature_y], c=y[0:1000], cmap="viridis")
    plt.xlabel(f'Feature {feature_x}')
    plt.ylabel(f'Feature {feature_y}')
    

    cbar = plt.colorbar(sc)
    cbar.set_label("Label")
    plt.show()

scatter_plot(11,4,3)

# =========================
# Partie 2 : Feature Engineering
# =========================

In [ ]:
# =============================
# Réduction de features
# =============================
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD

print("\n--- Réduction des features ---")

path =f"url_svmlight/url_svmlight/Day96.svm"
X, y = load_svmlight_file(path)

# (a) Supprimer les colonnes complètement nulles
vt = VarianceThreshold(threshold=0.0)
X_reduced = vt.fit_transform(X)

print("Shape après suppression des colonnes nulles :", X_reduced.shape)

# (b) Supprimer les features très rares (optionnel mais utile)
print("Suppression des features très rares (présentes < 5 fois)...")
X_csc = X_reduced.tocsc()
feature_counts = np.diff(X_csc.indptr)
mask = feature_counts >= 5
X_reduced = X_csc[:, mask].tocsr()
print("Shape après suppression des features rares :", X_reduced.shape)

print("\n--- PCA ---")
svd = TruncatedSVD(n_components=2)
X_svd = svd.fit_transform(X)
plt.scatter(X_svd[:,0], X_svd[:,1], c=y, cmap='coolwarm', s=10, alpha=0.6)
plt.xlabel('Composante 1')
plt.ylabel('Composante 2')
plt.title('Projection des URLs sur les 2 premières composantes SVD')
plt.colorbar(label='label')
plt.show()

Nous commençons par une pca brute pour voir ce que cela peut donner. 

**PCA**

In [ ]:
from sklearn.decomposition import TruncatedSVD

X, y = load_svmlight_file('url_svmlight/url_svmlight/Day96.svm')

svd = TruncatedSVD(n_components=2)
X_svd = svd.fit_transform(X)
# print(X_svd.shape)


In [ ]:
print(svd.explained_variance_ratio_)
plt.scatter(X_svd[:1000,0], X_svd[:1000,1], c=y[:1000], cmap='coolwarm', s=10, alpha=0.6)
plt.xlabel('Composante 1')
plt.ylabel('Composante 2')
plt.title('Projection des URLs sur les 2 premières composantes SVD')
plt.colorbar(label='label')
plt.show()

In [ ]:
A = np.zeros((2,3))
print(A)

Constatant l'efficacité toute relative de cette PCA, nous décidons de retravailler sur les données d'entrées afin d'éliminer dès le départ des features à trop faible variance.
Nous remarquons que beaucoup de colonnes sont nulles sur la prévisualisation, il faut les retirer du dataset. 
La difficulté est de parcourir tous les fichiers svm. 

Comme beaucoup de nos colonnes de features ne contiennent que des 0 ou des 1, et que cela ne permettra pas la classification, nous décidons d'enlever ces colonnes dans les données qui serveront à l'apprentissage. 

Pour ce faire, nous utilisons un critère sur la variance minimale d'une colonne dans chaque fichier. Puis, nous regardons le nombre de fichier pour lesquels une colonne a été gardée. Nous mesurons cela en pourcentage. Exemple : la colonne 1 a une variance supérieure au critère minimal dans les fichiers 1 à 10, mais pas dans les fichiers 11 à 20. Ainsi, l'algorithme a gardé la colonne un pour les fichiers 1 à 10 et l'a enlevée dans les autres. Au total, la colonne 1 a été gardée dans 50% des cas.

Dans un premier temps, nous gardons les colonnes dès lors qu'elles sont gardées au moins une fois, soit que leur pourcentage d'apparition est strictement positif. Nous pourrons raffiner cela pour garder moins de features si on voit que cela améliore la performance de la méthode d'apprentissage.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
import os

data_dir = "url_svmlight/url_svmlight"
X_list, y_list = [], []



selector = VarianceThreshold(threshold=0.01)

for file in sorted(os.listdir(data_dir))[:10]:  # exemple sur 10 jours
    X, y = load_svmlight_file(os.path.join(data_dir, file))
    # print('passage')
    X_list.append(X)
    y_list.append(y)


kept_mask = []
for i in range(len(X_list)):
    X_reduced = selector.fit_transform(X_list[i])
    print(X_list[i].shape, "→", X_reduced.shape)
    keep_mask = selector.get_support()
    kept_mask.append(np.where(keep_mask)[0])


In [2]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import load_svmlight_file
import os

data_dir = "url_svmlight/url_svmlight"
X_list, y_list = [], []

max_features = 3300000  # à adapter à ton dataset

selector = VarianceThreshold(threshold=0.01)
kept_mask_list = []

# Boucle sur les fichiers
for file in sorted(os.listdir(data_dir))[:10]:  # exemple sur 10 jours
    X, y = load_svmlight_file(os.path.join(data_dir, file), n_features=max_features)
    X_list.append(X)
    y_list.append(y)

    X_reduced = selector.fit_transform(X)
    keep_mask = selector.get_support()  # booléen : True si la colonne est gardée
    kept_mask_list.append(keep_mask)

# Transformer en matrice 2D : fichiers × colonnes
kept_mask_matrix = np.array(kept_mask_list, dtype=int)

# Pourcentage de fois où chaque colonne est gardée
column_keep_percentage = kept_mask_matrix.mean(axis=0) * 100

# Créer DataFrame en ne gardant que les colonnes qui ont été sélectionnées au moins une fois
df_keep = pd.DataFrame({
    'column_index': np.arange(len(column_keep_percentage)),
    'percent_kept': column_keep_percentage
})

# Filtrer les colonnes jamais gardées
df_keep = df_keep[df_keep['percent_kept'] > 0]
# Trier par pourcentage décroissant si tu veux
# df_keep = df_keep.sort_values(by='percent_kept', ascending=False).reset_index(drop=True)

# Afficher un aperçu
# print(df_keep.head(30))



In [5]:
import os
from sklearn.datasets import load_svmlight_files
from sklearn.feature_selection import VarianceThreshold
from scipy.sparse import vstack

data_dir = "url_svmlight/url_svmlight"

# Liste complète des chemins
files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir)])[:2]  # ou plus

# Lecture en une seule fois : bien plus rapide que 10 load_svmlight_file
X_all, y_all = load_svmlight_files(files, n_features=3_300_000)

# Si load_svmlight_files renvoie plusieurs sorties, les empiler :
if isinstance(X_all, tuple):
    X_all = vstack(X_all)
    y_all = np.hstack(y_all)

# Sélection de variance
selector = VarianceThreshold(threshold=0.01)
X_reduced = selector.fit_transform(X_all)

keep_mask = selector.get_support()
kept_indices = np.where(keep_mask)[0]

print(f"{keep_mask.sum()} variables conservées sur {keep_mask.size}")

# Si tu veux conserver le pourcentage ou l’importance
variances = selector.variances_
df_keep = pd.DataFrame({
    'column_index': np.arange(len(variances)),
    'variance': variances,
    'kept': keep_mask
})


ValueError: too many values to unpack (expected 2)

In [9]:
from sklearn.datasets import load_svmlight_file
import numpy as np
import os

data_dir = "url_svmlight/url_svmlight"
files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir)])[:20]

max_features = 3_300_000
mean_ = np.zeros(max_features)
mean_sq_ = np.zeros(max_features)
n_total = 0

for file in files:
    X, _ = load_svmlight_file(file, n_features=max_features)
    n_samples = X.shape[0]
    mean_ += X.mean(axis=0).A1 * n_samples
    mean_sq_ += (X.power(2).mean(axis=0).A1) * n_samples
    n_total += n_samples

mean_ /= n_total
mean_sq_ /= n_total
variances = mean_sq_ - mean_**2
keep_mask = variances > 0.01


In [10]:
print(keep_mask)

[False  True False ... False False False]


Sélection des données d'entraînement :

In [3]:
import scipy.sparse as sp

selected_columns = df_keep['column_index'].values

X_filtered_list, y_filtered_list = [],[]

for file in sorted(os.listdir(data_dir))[:10]:
    X,y = load_svmlight_file(os.path.join(data_dir, file), n_features=max_features)

    X_filtered = X[:,selected_columns]
    X_filtered_list.append(X_filtered)
    y_filtered_list.append(y)


X_all = sp.vstack(X_filtered_list)
y_all = np.concatenate(y_filtered_list)

**PCA sur les données filtrées**

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# X_reduced_scaled = StandardScaler().fit_transform(X_reduced)

# pca = PCA(n_components=2)
# pca.fit(X_reduced_scaled)
# X_reduced_proj = pca.transform(X_reduced_scaled)
# print(pca.explained_variance_ratio_)

# plt.figure(figsize=(15,5))
# plt.subplot(1,2,1)
# plt.scatter(X_reduced[:,15], X_reduced[:,16],c = y, cmap='viridis', alpha=0.5)
# plt.grid(alpha=0.2)
# plt.title('Dans les coordonées de base')
# plt.colorbar(label='label')

# plt.subplot(1,2,2)
# plt.scatter(X_reduced_proj[:,0], X_reduced_proj[:,1], c = y, cmap='viridis', alpha=0.5)
# plt.xlabel('Première composante principale')
# plt.ylabel('Deuxième composante principale')
# plt.colorbar(label='label')
# plt.grid(alpha=0.2)

# plt.show()



# =========================
# Partie 3 : Phase d'apprentissage
# =========================

In [ ]:
#SVM
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from time import time


# ⚠️ Important : un kernel RBF ne gère pas directement les matrices sparse.
# On convertit donc un échantillon en dense (en RAM attention)
# Si ton dataset est trop gros, on prend un sous-échantillon.
if X_reduced.shape[0] > 5000 or X_reduced.shape[1] > 5000:
    print("Dataset trop volumineux — on prend un échantillon de 5000 URLs pour la démonstration.")
    from sklearn.utils import resample
    X_reduced, y = resample(X_reduced, y, n_samples=5000, random_state=42)
    X_reduced = X_reduced.toarray()  # conversion en dense
else:
    X_reduced = X_reduced.toarray()

# =========================
# 2️⃣ Séparer train/test
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42, stratify=y
)

# Normalisation recommandée pour SVM RBF
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# =========================
# 3️⃣ Entraînement SVM (kernel trick)
# =========================
print("\n--- Entraînement du SVM avec noyau RBF ---")

svm_model = SVC(
    kernel='rbf',     # 'linear', 'poly', 'rbf', 'sigmoid'...
    C=1.0,            # paramètre de régularisation
    gamma='scale',    # influence du noyau RBF (auto ou 'scale')
    class_weight='balanced',  # utile si les classes sont déséquilibrées
)

t0 = time()
svm_model.fit(X_train, y_train)
t1 = time()

print(f"✅ Modèle entraîné en {t1 - t0:.2f} secondes")

# =========================
# 4️⃣ Évaluation
# =========================
y_pred = svm_model.predict(X_test)

print("\n--- Évaluation du modèle ---")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("\nMatrice de confusion :\n", confusion_matrix(y_test, y_pred))
print("\nRapport de classification :\n", classification_report(y_test, y_pred))


**Revue des méthodes qui pourraient être utiles**

Nous pouvons déjà rejeter le bagging, car notre jeu de données contient largement assez d'échantillons. Il reste svm, l'inférence bayésienne, la régression logistique (si multi-linéarité), les arbres de décisions (XGboost), les processsus gaussiens.

Commençons par un svm naïf. 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Entraînement pur :

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


svm = LinearSVC(random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_test_scaled)

print('Précision de : ', accuracy_score(y_test, y_pred))
print("\nRésumé de classification:\n", classification_report(y_test, y_pred))
print("\nMatrice de confusion:\n", confusion_matrix(y_test,y_pred))

Tentative de validation croisée : 

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    MaxAbsScaler(),           # scaler rapide pour sparse matrices
    LinearSVC(
        dual=False,           # plus rapide si n_samples > n_features
        max_iter=3000,        # limite d'itérations
        tol=1e-3,             # tolérance plus élevée pour converger plus vite
        random_state=42
    )
)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_all, y_all, cv=4, n_jobs=-1)

print("Scores sur les folds :", scores)
print("Précision moyenne :", np.mean(scores))
print("Écart-type :", np.std(scores))

**SVM avec un kernel non linéaire**

In [ ]:

from sklearn.svm import SVC
# Pipeline pour gérer le sparse matrix
model = make_pipeline(
    MaxAbsScaler(),                # scaler rapide compatible sparse
    SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)  # kernel RBF
)

# Cross-validation sur un échantillon réduit pour tester
sample_idx = np.random.choice(X_all.shape[0], size=100_000, replace=False)
X_sample = X_all[sample_idx]
y_sample = y_all[sample_idx]

scores = cross_val_score(model, X_sample, y_sample, cv=3, n_jobs=-1)
print("Scores CV :", scores)
print("Précision moyenne :", np.mean(scores))
print("Écart-type :", np.std(scores))


Nous constatons une légère amélioration de la précision obtenue avec un kernel rbf. Néanmoins, le temps de calcul est nettement plus long. Vu la taille du jeu de données qu'on souhaite tester, il n'est pas envisageable de prendre le rbf tel quel. Il pourrait convenir si on avait moins de features. Mais, dans ce cas, n'aurait-on pas un soucis de nombre de samples par rapport au nombre de features ? 

Etant donné qu'on obtient des résultats satisfaisants avec le svm linéaire, une bonne précision, sans surapprentissage (nous l'avons vérifié avec la validation croisée), il paraît assez optimal de continuer avec le svm linéaire. Quel paramètre peut-on ajuster pour la suite ? 

Peut-être faut-il interroger le nombre de features sélectionné avec le critère sur la variance ? Nous souhaitions aussi implémenter une autre technique de sélection de variable consistant à évaluer la pertinence d'une feature sur la prédiction de classe du sample (avec un test du chi2 par exemple). 

**Test de la régression logisitique :**

Etant donné que le svm linéaire donne des résultats satisfaisants, essayons une méthode de classification linéaire qui a l'avantage de pouvoir gérer de grands volumes de données, ce qui est notre cas ici. 

*Régression L1 pour sélectionner les features les plus prédictives*

L'idée est de garder le moins de features possible par régression L1 pour accélerer l'execution de la régression L2. 

In [ ]:
# from sklearn.linear_model import LogisticRegression

# clf_l1 = LogisticRegression(
#     penalty='l1',
#     solver='saga',      # gère sparse et grands datasets
#     max_iter=500,        # ajustable pour la vitesse
#     C=0.1,               # régularisation forte → plus de coefficients à zéro
#     n_jobs=-1
# )
# clf_l1.fit(X_train_scaled, y_train)

In [ ]:
# coef = clf_l1.coef_[0]
# important_idx = np.where(coef != 0)[0]

# Si tu veux les noms des features (X_train doit être un DataFrame)
# important_features = np.array(X_train.columns)[important_idx]
# print("Nombre de features importantes :", len(important_features))


In [ ]:
# X_train_reduced = X_train_scaled[:, important_idx]
# X_test_reduced = X_test_scaled[:, important_idx]

In [ ]:
# clf_l2 = LogisticRegression(
#     penalty='l2',
#     solver='saga',
#     max_iter=1000,
#     n_jobs=-1
# )
# clf_l2.fit(X_train_reduced, y_train)


In [ ]:
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# y_pred = clf_l2.predict(X_test_reduced)
# y_prob = clf_l2.predict_proba(X_test_reduced)[:, 1]

# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("F1-score:", f1_score(y_test, y_pred))
# print("ROC-AUC:", roc_auc_score(y_test, y_prob))

Après mise en place de la méthode L1 puis L2, on se rend compte que la méthode L1 est plus lente que la L2 et donc qu'on perd plus de temps à sélectionner les variables prédictives en mettant les autres à 0 au moyen d'un régression L1 qu'à faire directement une régression L2.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    penalty='l2',       # ou 'l1' pour sélection de features
    solver='saga',      # efficace pour grands datasets
    max_iter=1000,      # augmenter si convergence lente
    n_jobs=-1           # parallélisation
)

clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, clf.predict_proba(X_test_scaled)[:,1]))


**Inférence Bayésienne**

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, roc_auc_score

# alpha = 1 correspond à un prior Beta(1,1)
model = BernoulliNB(alpha=1.0, binarize=None)  # binarize=None car tes features sont déjà binaires
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))

Accuracy: 0.9035969387755102
AUC: 0.9609527940606886


Approche par batch 

In [12]:
import os
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import scipy.sparse as sp

data_dir = "url_svmlight/url_svmlight"
max_features = 3_300_000
keep_mask = np.array(keep_mask)
selected_columns = np.where(keep_mask)[0]


# Modèle
model = BernoulliNB(alpha=1.0)
classes = np.array([-1,1])

# Split train/test
# Ici, on réserve le dernier fichier pour test
all_files = sorted([os.path.join(data_dir,f) for f in os.listdir(data_dir)])[:20]
train_files = all_files[:-1]
test_file = all_files[-1]

# --- Entraînement batch par batch ---
for file in train_files:
    X, y = load_svmlight_file(file, n_features=max_features)
    X_filtered = X[:, selected_columns]
    model.partial_fit(X_filtered, y, classes=classes)

# --- Evaluation sur le fichier test ---
X_test, y_test = load_svmlight_file(test_file, n_features=max_features)
X_test_filtered = X_test[:, selected_columns]

y_pred = model.predict(X_test_filtered)
y_proba = model.predict_proba(X_test_filtered)[:,1]

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy : {acc:.4f}")
print(f"AUC : {auc:.4f}")
print("Matrice de confusion :\n", cm)


Accuracy : 0.8982
AUC : 0.9589
Matrice de confusion :
 [[12383   513]
 [ 1522  5582]]


# =========================
# Partie 4 : Tuning d'un hyperparamètre
# =========================

In [ ]:
#Code

# =========================
# Partie 5 : Conclusions
# =========================

In [ ]:
#Code

In [ ]:
path =f"url_svmlight/url_svmlight/Day1.svm"
X, y = load_svmlight_file(path)
print(X)
print(y)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.datasets import load_svmlight_file
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import shuffle
from time import time

# =============================
# 1️⃣ Charger le fichier .svm
# =============================
print("Chargement du dataset...")
X, y = load_svmlight_file(path)  # <-- à adapter à ton chemin
print(f"Shape initial : {X.shape}")
print(f"Nombre d'éléments non nuls : {X.nnz}")
print(f"Taux de sparsité : {100*(1 - X.nnz/(X.shape[0]*X.shape[1])):.6f}%")

# Mélanger pour éviter un ordre biaisé
X, y = shuffle(X, y, random_state=42)

# =============================
# 2️⃣ Séparer train/test
# =============================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# =============================
# 3️⃣ Entraînement avant nettoyage
# =============================
print("\n--- Entraînement AVANT réduction ---")
clf = LogisticRegression(max_iter=1000)
t0 = time()
clf.fit(X_train, y_train)
t1 = time()
y_pred = clf.predict(X_test)
print(f"Temps d'entraînement : {t1 - t0:.2f}s")
print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# =============================
# 4️⃣ Réduction de features
# =============================
print("\n--- Réduction des features ---")

# (a) Supprimer les colonnes complètement nulles
vt = VarianceThreshold(threshold=0.0)
X_reduced = vt.fit_transform(X)

print("Shape après suppression des colonnes nulles :", X_reduced.shape)

# (b) Supprimer les features très rares (optionnel mais utile)
print("Suppression des features très rares (présentes < 5 fois)...")
X_csc = X_reduced.tocsc()
feature_counts = np.diff(X_csc.indptr)
mask = feature_counts >= 5
X_reduced = X_csc[:, mask].tocsr()
print("Shape après suppression des features rares :", X_reduced.shape)

# =============================
# 5️⃣ Réentraînement après réduction
# =============================
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

print("\n--- Entraînement APRÈS réduction ---")
clf2 = LogisticRegression(max_iter=1000)
t0 = time()
clf2.fit(X_train, y_train)
t1 = time()
y_pred = clf2.predict(X_test)
print(f"Temps d'entraînement : {t1 - t0:.2f}s")
print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

print("\n✅ Script terminé avec succès.")
